In [1]:
from grid2op.Agent import BaseAgent
from grid2op.Agent import DoNothingAgent
from grid2op import make
from grid2op.PlotGrid import PlotMatplot
from DenseNN import DenseNN
from DQN import DQN
import numpy as np
from evaluate import evaluate
import os

In [10]:
env = make("l2rpn_neurips_2020_track2_small")
plot_helper = PlotMatplot(env.observation_space)

In [7]:
class DQNAgent(BaseAgent):
    """
    The template to be used to create an agent: any controller of the power grid is expected to be a subclass of this
    grid2op.Agent.BaseAgent.
    """
    def __init__(self, env, curr_dir):
      """Initialize a new agent."""
      BaseAgent.__init__(self, action_space=env.action_space)

      self.all_actions = [env.action_space({})]
      actions = np.load(os.path.join(curr_dir, "top1000_actions.npz"), allow_pickle=True)["actions"]
      for action in actions:
        self.all_actions.append(env.action_space.from_vect(action))
      self.all_actions = np.asarray(self.all_actions)

      self.dqn = DQN(self, (lambda : DenseNN(env.current_obs.to_vect().shape, self.all_actions.shape[0])))

    def act(self, observation, reward, done):
      """The action that your agent will choose depending on the observation, the reward, and whether the state is terminal"""
      # do nothing for example (with the empty dictionary) :
      
      return self.dqn.select_action(observation)

    def train(self, env, nb_episode):
      self.dqn.replay_exp(env, nb_episode=nb_episode)
    

class RandomAgent(BaseAgent):
    """
    The template to be used to create an agent: any controller of the power grid is expected to be a subclass of this
    grid2op.Agent.BaseAgent.
    """
    def __init__(self, env, curr_dir):
      """Initialize a new agent."""
      BaseAgent.__init__(self, action_space=env.action_space)

      self.all_actions = [env.action_space({})]
      actions = np.load(os.path.join(curr_dir, "top1000_actions.npz"), allow_pickle=True)["actions"]
      for action in actions:
        self.all_actions.append(env.action_space.from_vect(action))
      self.all_actions = np.asarray(self.all_actions)

    def act(self, observation, reward, done):
      """The action that your agent will choose depending on the observation, the reward, and whether the state is terminal"""
      # do nothing for example (with the empty dictionary) :
      
      return np.random.choice(self.all_actions)


    
def make_agent(env, this_directory_path):
  my_agent = DQNAgent(env, this_directory_path)
  return my_agent

In [8]:
dqn_agent = DQNAgent(env, ".")
nothing_agent = DoNothingAgent(env.action_space)
dqn_agent.train(env, 300)

<bound method GridObjects.shape of <grid2op.Space.GridObjects.CompleteObservation_l2rpn_neurips_2020_track2_x1 object at 0x7f0dbeba6640>>
Episode 0 -> survived steps: 7 total reward: 3576.52
Episode 1 -> survived steps: 9 total reward: 6255.60
Episode 2 -> survived steps: 11 total reward: 6713.71
Episode 3 -> survived steps: 20 total reward: 16955.52
Episode 4 -> survived steps: 7 total reward: 3330.40


KeyboardInterrupt: 

In [ ]:
env = make("l2rpn_neurips_2020_track2_small")
evaluate(dqn_agent, env)

In [ ]:
env = make("l2rpn_neurips_2020_track2_small")
evaluate(nothing_agent, env)

In [ ]:
env.reset()
print(env.name)
obs = env.current_obs
reward = env.current_reward
done = False
actions = [env.action_space({}), env.action_space({})]
sum_r = 0
for i in range(25):
  action = dqn_agent.act()
  obs, reward, done, info = env.step(action)
  sum_r += reward
  print(reward)
  print(done)
  if done:
    break
print("Sum", sum_r)